In [159]:
# run this to read in the rent registry

import pandas as pd
import numpy as np

fn = 'Rent Registry_PRA 22-496.xlsx'
#fn = 'test.xlsx'
ccrr = pd.read_excel(fn)
ccrr['AddressLine1'] = ccrr['AddressLine1'].str.replace('10144 CULVER CITY','10144 CULVER BLVD')
ccrr = ccrr.sort_values(by='PastRentReportDate',ascending=False).reset_index()

In [76]:
ccrr.drop_duplicates(['AddressLine1','UnitNumb'])[['AddressLine1','UnitNumb']]

,AddressLine1,UnitNumb
0,9770 CULVER BLVD,111
2,5127 FAIRBANKS WAY,1
6,3573 HELMS AVE,3573
10,5830 GREEN VALLEY CIR,101
13,5830 GREEN VALLEY CIR,102
...,...,...
22299,5900 CANTERBURY DR UNIT A-210,NaN
22300,11225 MCDONALD ST,NaN
22301,14303 SUMMERTIME LN,NaN
22302,11036 CULVER BLVD,NaN


In [ ]:
# run this to geocode all the addresses in the rent registry

In [158]:
# run this to create the json that contains the data to be displayed

import usaddress
import datetime
import time
import orjson

export_json = dict()

for index, row in ccrr.drop_duplicates(['AddressLine1','UnitNumb']).iterrows():
    #TODO: figure out the structure of your JSON
    # I think we want it to be a dictionary of AddressLine1's (and separately we've got a csv that has three columns: addressline1, long, lat)
    # then within that there's a dictionary of features... one of them is a list of units
    # each unit is a dictionary that stores info on a unit, rent history, current rent, beds/baths, etc.
    # OR do we want it stuctured closer to how I want to display it?
    # OR is it more flat so I can use database calls from the JavaScript itself? (i.e., select the 3 beds so we can group by beds when we display?)
    # MAYBE this is creating a JSON that can be displayed on my map and separately I have function to revamp the data for analysis
    # 
    if pd.isna(row['AddressLine1']):
        continue
    
    line1 = row['AddressLine1']
    line2 = row['AddressLine2']
    unit = row['UnitNumb']
    beds = row['NumofBeds']
    baths = row['NumOfBaths']
    built = row['YearBuilt']
    rent = row['RentAmount']
    if pd.isna(row['PastRentReportDate']):
        rentdate = ''
    else:
        rentdate = row['PastRentReportDate'].strftime('%Y/%m/%d')
    encumbered = row['EmcumberedUnit']
    status = row['Unit Status']
    
    if isinstance(line1,str) and '3959 AND 3961 BENTLEY AVE' in line1:
        line1 = '3959-3961 BENTLEY AVE'
    elif isinstance(line1,str) and '4222 & 4224 JASMINE AVE' in line1:
        line1 = '4222-4224 JASMINE AVE'
    elif isinstance(line1,str) and '4070 LAFAYETTE PL # House B' in line1:
        line1 = '4070 LAFAYETTE PL House B'
    elif isinstance(line1,str) and 'Do not own a rental property in Culver City' in line1:
        continue
    if pd.isna(line2):
        line2 = ''
    tagged_address, address_type= usaddress.tag(str(line1)+', '+str(line2))

    if ('StreetNamePostType' or 'StreetName' or 'AddressNumber') not in tagged_address.keys():
        print(tagged_address)
    if 'StreetNamePostType' not in tagged_address.keys():
        address = ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName']])
    else:
        address = ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName'], tagged_address['StreetNamePostType']])
    
    if address not in export_json.keys():
        export_json[address] = []
    export_json[address].append([unit, beds, baths, built, rent, rentdate, encumbered, status])

with open("ccrr-" + time.strftime("%Y%m%d-%H%M%S") + ".json", "wb") as f:
    f.write(orjson.dumps(export_json))

OrderedDict([('AddressNumber', '6124'), ('StreetName', 'BUCKINGHAM PKWY 19'), ('PlaceName', 'CULVER CITY'), ('StateName', 'CA'), ('ZipCode', '90230-7222')])
OrderedDict([('AddressNumber', '5014'), ('StreetName', 'RAINBOWS END'), ('PlaceName', 'CULVER CITY'), ('StateName', 'CA'), ('ZipCode', '90230-4471')])
OrderedDict([('AddressNumber', '5010'), ('StreetName', 'RAINBOWS END'), ('PlaceName', 'CULVER CITY'), ('StateName', 'CA'), ('ZipCode', '90230-4471')])
OrderedDict([('AddressNumber', '5003'), ('StreetName', 'RAINBOWS END'), ('PlaceName', 'CULVER CITY'), ('StateName', 'CA'), ('ZipCode', '90230-4472')])


In [134]:
tagged_address.keys()

odict_keys(['AddressNumber', 'StreetName', 'StreetNamePostType', 'PlaceName', 'StateName', 'ZipCode'])

In [115]:
import usaddress
import time
import datetime
import orjson 

line1 = '4900  OVERLAND AVE  UNIT 396'
line2 = 'CULVER CITY, CA 90230-4299'
unit = 396
beds = 2
baths = 2
built = 1973
rent = 2725
rentdate = datetime.datetime.strptime('06/18/2022','%m/%d/%Y').date()
encumbered = "No"
status = "Rented"
export_json = dict()

tagged_address, address_type= usaddress.tag(line1+', '+line2)

address = ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName'], tagged_address['StreetNamePostType']])
export_json[address] = [unit, beds, baths, built, rent, rentdate, encumbered, status]

with open("ccrr-" + time.strftime("%Y%m%d-%H%M%S") + ".json", "wb") as f:
    f.write(orjson.dumps(export_json))


In [114]:
time.strftime("%Y%m%d-%H%M%S")

'20230129-201338'

In [94]:
outliers = pd.DataFrame(columns=list(ccrr.columns))
for index, row in ccrr.drop_duplicates(['AddressLine1','UnitNumb'])[['AddressLine1','UnitNumb']].iterrows():
    address = row['AddressLine1']
    unit_number = row['UnitNumb']
    # todo: print address and unit_number, wait 15minutes and see what's crashing this next line
    subset = ccrr[(ccrr['AddressLine1'].str.contains(address)) & ((ccrr['UnitNumb']==unit_number)|(ccrr['UnitNumb'].str.contains(str(unit_number)))) & (ccrr['PastRentReportDate'].notna())]
    subset = subset.sort_values(by='PastRentReportDate',ascending=False).reset_index()
    a = subset
    if subset.empty:
        continue
    if (subset['PastAmount'].iloc[0] != subset['RentAmount'].iloc[0]):
        outliers = pd.concat([outliers,subset.loc[[0]]])
print(outliers)

<ipython-input-94-541f0dd35c04>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset = ccrr[(ccrr['AddressLine1'].str.contains(address)) & ((ccrr['UnitNumb']==unit_number)|(ccrr['UnitNumb'].str.contains(str(unit_number)))) & (ccrr['PastRentReportDate'].notna())]


error: nothing to repeat at position 0

In [95]:
outliers

,AddressLine1,AddressLine2,UnitNumb,NumofBeds,NumOfBaths,YearBuilt,RentAmount,PastRentReportDate,PastAmount,EmcumberedUnit,Unit Status


In [2]:
from mapbox import Geocoder

geocoder = Geocoder(access_token='pk.eyJ1IjoiaHVnaGZpdHpnZXJhbGQiLCJhIjoiY2xkZGhjaG9wMDNqdTNvdDZ5bG80OXZ3YSJ9.6njmVnJyl0zAtMnM9d8duQ')
response = geocoder.forward('4252 Coolidge Ave, Los Angeles, CA 90066')

<Response [200]>


In [6]:
response.json()['features'][0]

{'id': 'address.7082831410212634',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Coolidge Avenue',
 'place_name': '4252 Coolidge Avenue, Los Angeles, California 90066, United States',
 'center': [-118.4171, 34.00101],
 'geometry': {'type': 'Point', 'coordinates': [-118.4171, 34.00101]},
 'address': '4252',
 'context': [{'id': 'neighborhood.394022124', 'text': 'McLaughlin'},
  {'id': 'postcode.300560108', 'text': '90066'},
  {'id': 'place.192407788', 'wikidata': 'Q65', 'text': 'Los Angeles'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}

In [7]:
address = '6605  GREEN VALLEY CIR  UNIT 113	CULVER CITY, CA 90230-8083'
response = geocoder.forward(address)
response.json()['features'][0]

{'id': 'address.5063949685794286',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Green Valley Circle',
 'place_name': '6605 Green Valley Circle, Culver City, California 90230, United States',
 'center': [-118.385796, 33.97883],
 'geometry': {'type': 'Point', 'coordinates': [-118.385796, 33.97883]},
 'address': '6605',
 'context': [{'id': 'neighborhood.229092588', 'text': 'Fox Hills'},
  {'id': 'postcode.300904172', 'text': '90230'},
  {'id': 'place.77531372', 'wikidata': 'Q493378', 'text': 'Culver City'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}

In [9]:
import usaddress
usaddress.parse(address)

[('6605', 'AddressNumber'),
 ('GREEN', 'StreetName'),
 ('VALLEY', 'StreetName'),
 ('CIR', 'StreetNamePostType'),
 ('UNIT', 'OccupancyType'),
 ('113', 'OccupancyIdentifier'),
 ('CULVER', 'PlaceName'),
 ('CITY,', 'PlaceName'),
 ('CA', 'StateName'),
 ('90230-8083', 'ZipCode')]

In [10]:
address = '9770 CULVER BLVD	CULVER CITY, CA 90232-2703'
response = geocoder.forward(address)
response.json()['features'][0]

{'id': 'address.766468067844202',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Culver Boulevard',
 'place_name': '9770 Culver Boulevard, Culver City, California 90232, United States',
 'center': [-118.39559, 34.02163],
 'geometry': {'type': 'Point', 'coordinates': [-118.39559, 34.02163]},
 'address': '9770',
 'context': [{'id': 'neighborhood.375500012', 'text': 'Lucerne'},
  {'id': 'postcode.300920556', 'text': '90232'},
  {'id': 'place.77531372', 'wikidata': 'Q493378', 'text': 'Culver City'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}

In [11]:
address = '4396 KEYSTONE AVE	CULVER CITY, CA 90232-3437'
response = geocoder.forward(address)
response.json()['features'][0]

{'id': 'address.1144368606704674',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Keystone Avenue',
 'place_name': '4396 Keystone Avenue, Culver City, California 90232, United States',
 'center': [-118.39549, 34.009255],
 'geometry': {'type': 'Point', 'coordinates': [-118.39549, 34.009255]},
 'address': '4396',
 'context': [{'id': 'neighborhood.91180268', 'text': 'Carlson'},
  {'id': 'postcode.300920556', 'text': '90232'},
  {'id': 'place.77531372', 'wikidata': 'Q493378', 'text': 'Culver City'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}